In [70]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

โหลดข้อมูลจากไฟล์ CSV เป็น DataFrame

In [71]:
df = pd.read_csv('student.csv')

แสดงข้อมูล 5 แถวแรก เพื่อดูโครงสร้างข้อมูลเบื้องต้น

In [72]:
df.head(5)

,รหัสนักศึกษา,ชื่อ,คณะ,ชั้นปี,GPA,กิจกรรม,เวลาเข้าร่วม
0,64001,สมชาย,วิทยาศาสตร์,2.0,3.2,กีฬา,08:00
1,64002,สมหญิง,ศิลปศาสตร์,3.0,3.8,ดนตรี,9.30
2,64003,NaN,วิทยาศาสตร์,2.0,-1.0,"กีฬา,ดนตรี",08:30
3,64004,สมศักดิ์,ศิลปศาสตร์,NaN,2.9,NaN,08:00
4,64005,สมใจ,วิศวกรรมศาสตร์,4.0,4.5,อาสา,NaN


ตรวจสอบจำนวนแถวก่อนและหลังลบแถวที่มีค่า NaN

In [73]:
print("\nจำนวนแถวก่อนลบค่า NaN:", len(df))
df_dropna = df.dropna()  # ลบแถวที่มีค่า NaN ในคอลัมน์ใดคอลัมน์หนึ่งเลย
print("จำนวนแถวหลังลบค่า NaN:", len(df_dropna))
print("ข้อมูลหลังลบค่า NaN:")
df_dropna


จำนวนแถวก่อนลบค่า NaN: 5
จำนวนแถวหลังลบค่า NaN: 2
ข้อมูลหลังลบค่า NaN:


,รหัสนักศึกษา,ชื่อ,คณะ,ชั้นปี,GPA,กิจกรรม,เวลาเข้าร่วม
0,64001,สมชาย,วิทยาศาสตร์,2.0,3.2,กีฬา,08:00
1,64002,สมหญิง,ศิลปศาสตร์,3.0,3.8,ดนตรี,9.30


เติมค่า NaN ในคอลัมน์ชั้นปีด้วยค่าเฉลี่ยของชั้นปีทั้งหมด โดยค่าเฉลี่ยจะถูกปัดเป็นจำนวนเต็ม (ไม่ใช้ทศนิยม)

In [74]:
mean_year = round(df['ชั้นปี'].mean())  # คำนวณค่าเฉลี่ยและปัดทศนิยม
df['ชั้นปี'].fillna(mean_year, inplace=True)  # เติมค่า NaN ด้วยค่าเฉลี่ยที่ปัดแล้ว
df['ชั้นปี'] = df['ชั้นปี'].astype(int)  # เปลี่ยนประเภทข้อมูลเป็นจำนวนเต็ม
print(f"\nค่าเฉลี่ยชั้นปี: {mean_year}")


ค่าเฉลี่ยชั้นปี: 3


เติมค่า NaN ในคอลัมน์ คณะด้วยค่าที่พบบ่อยที่สุด (mode)

In [75]:
most_faculty = df['คณะ'].mode()[0]  # หา mode ของคณะ (ค่าที่พบมากที่สุด)
df['คณะ'].fillna(most_faculty, inplace=True)  # เติมค่า NaN ด้วย mode
print(f"\nคณะที่พบบ่อยที่สุด: {most_faculty}")


คณะที่พบบ่อยที่สุด: วิทยาศาสตร์


ทำให้ชื่อคณะเป็นมาตรฐาน เพื่อลดปัญหาการซ้ำซ้อนจากการพิมพ์ไม่เหมือนกัน

In [76]:
def standardize_faculty(name):
    # ตรวจสอบว่าชื่อที่รับเข้ามาอยู่ใน list ของชื่อที่เป็นไปได้หรือไม่
    if name in ['วิทยาศาสตร์', 'วิทย์', 'Science', 'คณะวิทยาศาสตร์']:
        return 'วิทยาศาสตร์'  # คืนค่าชื่อคณะที่เป็นมาตรฐาน
    elif name in ['ศิลปศาสตร์', 'ศิลป์', 'Arts']:
        return 'ศิลปศาสตร์'
    elif name in ['วิศวกรรมศาสตร์', 'วิศวะ', 'Engineering']:
        return 'วิศวกรรมศาสตร์'
    else:
        return name  # ถ้าไม่ตรงกับเงื่อนไขใดๆ ให้คืนชื่อเดิม
df['คณะ_มาตรฐาน'] = df['คณะ'].apply(standardize_faculty)
df['คณะ_มาตรฐาน']

,คณะ_มาตรฐาน
0,วิทยาศาสตร์
1,ศิลปศาสตร์
2,วิทยาศาสตร์
3,ศิลปศาสตร์
4,วิศวกรรมศาสตร์


ใช้วิธี IQR เพื่อตรวจจับและตัดค่า GPA ที่เป็นค่าผิดปกติ (Outliers) ออก

In [77]:
Q1 = df['GPA'].quantile(0.25)  # ควอร์ไทล์ที่ 25
Q3 = df['GPA'].quantile(0.75)  # ควอร์ไทล์ที่ 75
IQR = Q3 - Q1

กำหนดขอบเขตค่าปกติ โดยใช้ 1.5 เท่าของ IQR

In [78]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(f"\nGPA Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"ขอบเขตล่างของ GPA: {lower_bound}")
print(f"ขอบเขตบนของ GPA: {upper_bound}")


GPA Q1: 2.9, Q3: 3.8, IQR: 0.8999999999999999
ขอบเขตล่างของ GPA: 1.55
ขอบเขตบนของ GPA: 5.1499999999999995


กรองข้อมูลให้ GPA อยู่ในช่วงปกติ (ไม่มี outliers)

In [79]:
df = df[(df['GPA'] >= lower_bound) & (df['GPA'] <= upper_bound)]
print("ข้อมูลหลังตัดค่า GPA ที่เป็น Outliers:")
df[['GPA']].describe()

ข้อมูลหลังตัดค่า GPA ที่เป็น Outliers:


,GPA
count,4.000000
mean,3.600000
std,0.707107
min,2.900000
25%,3.125000
50%,3.500000
75%,3.975000
max,4.500000


แปลงข้อมูลคณะเป็นตัวเลขแบบ One-Hot Encoding

In [80]:
df = pd.get_dummies(df, columns=['คณะ_มาตรฐาน'])
print("ตัวอย่างข้อมูลที่ถูกแปลง One-Hot:")
df.head()

ตัวอย่างข้อมูลที่ถูกแปลง One-Hot:


,รหัสนักศึกษา,ชื่อ,คณะ,ชั้นปี,GPA,กิจกรรม,เวลาเข้าร่วม,คณะ_มาตรฐาน_วิทยาศาสตร์,คณะ_มาตรฐาน_วิศวกรรมศาสตร์,คณะ_มาตรฐาน_ศิลปศาสตร์
0,64001,สมชาย,วิทยาศาสตร์,2,3.2,กีฬา,08:00,True,False,False
1,64002,สมหญิง,ศิลปศาสตร์,3,3.8,ดนตรี,9.30,False,False,True
3,64004,สมศักดิ์,ศิลปศาสตร์,3,2.9,NaN,08:00,False,False,True
4,64005,สมใจ,วิศวกรรมศาสตร์,4,4.5,อาสา,NaN,False,True,False


แปลงข้อมูล 'เวลาเข้าร่วม' เป็นชนิด datetime

In [81]:
df['เวลาเข้าร่วม'] = pd.to_datetime(df['เวลาเข้าร่วม'], errors='coerce')

ดึงชั่วโมงจาก 'เวลาเข้าร่วม' เพื่อใช้แบ่งช่วงเวลา

In [82]:
df['ชั่วโมง'] = df['เวลาเข้าร่วม'].dt.hour

แบ่งช่วงเวลาเป็น 4 ช่วงตามชั่วโมงที่เข้าร่วมกิจกรรม

In [83]:
df['ช่วงเวลา'] = pd.cut(df['ชั่วโมง'],
                        bins=[-1, 10, 14, 18, 24],  # กำหนดขอบเขตช่วงเวลา
                        labels=['เช้า', 'กลางวัน', 'บ่าย', 'เย็น'])
df[['เวลาเข้าร่วม', 'ชั่วโมง', 'ช่วงเวลา']]

,เวลาเข้าร่วม,ชั่วโมง,ช่วงเวลา
0,2025-08-10 08:00:00,8.0,เช้า
1,NaT,NaN,NaN
3,2025-08-10 08:00:00,8.0,เช้า
4,NaT,NaN,NaN


สร้างคะแนนรวมโดยคำนวณจากการคูณชั้นปีและ GPA

In [84]:
df['คะแนนรวม'] = df['ชั้นปี'] * df['GPA']
df[['ชั้นปี', 'GPA', 'คะแนนรวม']]

,ชั้นปี,GPA,คะแนนรวม
0,2,3.2,6.4
1,3,3.8,11.4
3,3,2.9,8.7
4,4,4.5,18.0
